# 🧬 Tutorial 1: Data Extraction & Cleaning
## Working with ProteinGym DMS Data

---

**Learning Objectives:**
- Download and extract datasets from ProteinGym
- Clean and standardize tabular data with pandas
- Handle missing values and duplicates
- Normalize DMS scores for machine learning
- Visualize dataset characteristics

---

## 📥 Step 1: Data Download

We'll download the ProteinGym DMS (Deep Mutational Scanning) substitution dataset. Each CSV file represents one DMS experiment.

**Data Source:** [ProteinGym](https://proteingym.org/download) → DMS Assays → substitutions

In [ ]:
# Detect working directory and set base path for data files
# This handles two common execution scenarios:
# 1. Running from the notebook's directory (Tutorial_1_Data_Cleaning/)
# 2. Running from /workspace with repo at /workspace/tutorials

from pathlib import Path
import os
import urllib.request
import zipfile

cwd = Path.cwd()
target_dir = Path("ProteinGym_DMS_data") / "DMS_ProteinGym_substitutions"

# Define scenarios to check
scenarios = [
    {
        "name": "notebook directory",
        "check": lambda: cwd.name == "Tutorial_1_Data_Cleaning",
        "base": Path("..")
    },
    {
        "name": "/workspace",
        "check": lambda: cwd == Path("/workspace"),
        "base": Path("/workspace/tutorials")
    },
    {
        "name": "current directory",
        "check": lambda: True,
        "base": cwd
    },
    {
        "name": "fallback (parent directory)",
        "check": lambda: True,
        "base": Path("..")
    }
]

# Find the first scenario where data directory exists, or use fallback
BASE_DIR = None
for scenario in scenarios:
    if BASE_DIR is not None:
        break
    if not scenario["check"]():
        continue
    
    base_dir = scenario["base"]
    data_dir = base_dir / target_dir
    if data_dir.exists():
        BASE_DIR = base_dir
        scenario_name = scenario["name"]
        print(f"✓ Detected: Running from {scenario_name}")
        print(f"  Current directory: {cwd}")
        print(f"  Base directory: {BASE_DIR.resolve()}")
        break

# If no scenario found data, use the fallback (last scenario)
if BASE_DIR is None:
    BASE_DIR = scenarios[-1]["base"]
    print(f"⚠️  Warning: Unknown execution scenario, using fallback")
    print(f"  Current directory: {cwd}")
    print(f"  Base directory: {BASE_DIR.resolve()}")

print(f"\n📂 Using base directory: {BASE_DIR.resolve()}")

# Download and extract ProteinGym DMS substitution dataset if needed
data_dir = BASE_DIR / "ProteinGym_DMS_data" / "DMS_ProteinGym_substitutions"
zip_url = "https://marks.hms.harvard.edu/proteingym/ProteinGym_v1.3/DMS_ProteinGym_substitutions.zip"
zip_file = BASE_DIR / "ProteinGym_DMS_data" / "DMS_ProteinGym_substitutions.zip"

# Check if data already exists
if data_dir.exists() and data_dir.is_dir():
    csv_files = [f for f in data_dir.iterdir() if f.suffix == '.csv']
    if csv_files:
        print(f"\n✓ Data already downloaded!")
        print(f"  Found {len(csv_files)} CSV files in {data_dir}")
    else:
        print(f"\n⚠️  Directory exists but contains no CSV files. Downloading...")
        # Remove empty directory and download
        if data_dir.exists():
            data_dir.rmdir()
        download_and_extract = True
else:
    print(f"\n📥 Dataset not found. Downloading and extracting...")
    download_and_extract = True

# Download and extract if needed
if 'download_and_extract' in locals():
    # Create parent directory if it doesn't exist
    BASE_DIR.mkdir(exist_ok=True)
    (BASE_DIR / "ProteinGym_DMS_data").mkdir(exist_ok=True)
    
    # Download the zip file
    print(f"  Downloading from: {zip_url}")
    urllib.request.urlretrieve(zip_url, zip_file)
    print(f"  ✓ Downloaded to: {zip_file}")
    
    # Extract the zip file
    print(f"  Extracting archive...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(BASE_DIR / "ProteinGym_DMS_data")
    print(f"  ✓ Extracted successfully")
    
    # Remove the zip file
    zip_file.unlink()
    print(f"  ✓ Removed archive file")
    
    # Verify extraction
    csv_files = list(data_dir.glob('*.csv'))
    if csv_files:
        print(f"\n✓ Dataset ready!")
        print(f"  Found {len(csv_files)} CSV files in {data_dir}")
    else:
        print(f"\n⚠️  Warning: Extraction complete but no CSV files found in {data_dir}")

## 📊 Step 2: Load and Inspect Data

Now we'll load all CSV files and combine them into a single DataFrame.

In [ ]:
import pandas as pd
import numpy as np
import re

# Path to your DMS substitution data folder
data_dir = BASE_DIR / "ProteinGym_DMS_data" / "DMS_ProteinGym_substitutions"

# Load all CSVs into a list of dataframes
print("Loading CSV files...")
dfs = []
csv_files = list(data_dir.glob("*.csv"))

if not csv_files:
    print(f"⚠ No CSV files found at path: {data_dir}")
    print("  Please make sure the data has been downloaded first (run Step 1 cell above).")
else:
    for f in csv_files:
        df = pd.read_csv(f)
        df["source_file"] = f.stem  # Get filename without extension
        dfs.append(df)
    
    # Combine into a single dataframe
    data = pd.concat(dfs, ignore_index=True)
    
    print(f"✓ Loaded {len(dfs)} files with {len(data):,} total rows")

### 🔍 Quick Data Inspection

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
data.head()

In [ ]:
# Get data types and missing values info
print("Dataset information:")
data.info()

In [ ]:
# Count unique mutants
num_unique = data['mutant'].nunique()
print(f"Number of unique mutants: {num_unique:,}")

## 🧹 Step 3: Data Cleaning

### 3.1 Standardize Column Names

Let's rename columns to have consistent, clear names throughout our analysis.

In [ ]:
# Make column names consistent and easier to work with
df = data.rename(columns={
    'DMS_score': 'score',
    'mutant': 'mutation',
    'mutated_sequence': 'mut_seq',
    'DMS_score_bin': 'score_bin',
    'DMS_bin_score': 'score_bin_float'
})

print("✓ Column names standardized")
print(f"New columns: {list(df.columns)}")

### 3.2 Parse Mutation Strings

Extract wildtype amino acid, position, and mutant amino acid from mutation notation (e.g., "A123C").

In [ ]:
def parse_mut(m):
    """
    Parse mutation string (e.g., 'A123C') into components.
    Returns: (wildtype_aa, position, mutant_aa)
    """
    match = re.match(r"([A-Z])(\d+)([A-Z])", str(m))
    if match:
        wt, pos, mut = match.groups()
        return pd.Series([wt, int(pos), mut])
    return pd.Series([np.nan, np.nan, np.nan])

# Apply parsing to all mutations; this may take a moment
from tqdm import tqdm

# Use tqdm to add a progress bar to the apply function
tqdm.pandas(desc="Parsing mutations")
df[['wt_aa', 'position', 'mut_aa']] = df['mutation'].progress_apply(parse_mut)

print("✓ Mutations parsed successfully")
print(f"Example: {df[['mutation', 'wt_aa', 'position', 'mut_aa']].head(3)}")

### 3.3 Remove Invalid and Duplicate Entries

In [ ]:
# Store original size
original_size = len(df)

# Drop rows with missing values in critical columns
df = df.dropna(subset=['score', 'mutation', 'wt_aa', 'mut_aa'])

# Remove duplicate entries (same mutation in same source file)
df = df.drop_duplicates(subset=['source_file', 'mutation'])

print(f"✓ Removed {original_size - len(df):,} invalid/duplicate entries")
print(f"✓ Final dataset size: {len(df):,} rows")

## 📏 Step 4: Score Normalization

DMS scores can vary widely across different experiments. We'll normalize scores per experiment to help machine learning models generalize better.

**Why normalize?** Each experiment has its own scale. Standardizing to mean ≈ 0 and std ≈ 1 allows fair comparison across experiments.

### 4.1 Examine Score Distribution

In [ ]:
# Examine the raw score distribution
print("Raw DMS Score Statistics:")
print(f"  Maximum: {df['score'].max():.4f}")
print(f"  Minimum: {df['score'].min():.4f}")
print(f"  Mean:    {df['score'].mean():.4f}")
print(f"  Std Dev: {df['score'].std():.4f}")

### 4.2 Apply Z-Score Normalization

In [ ]:
# Normalize scores per experiment (z-score normalization)
df['score_norm'] = df.groupby('source_file')['score'].transform(
    lambda x: (x - x.mean()) / x.std()
)

print("✓ Scores normalized per experiment")

### 4.3 Verify Cleaned Dataset

In [ ]:
# Quick inspection of cleaned data
print(f"Dataset shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())

print("\nExperiment sizes:")
df.groupby('source_file').size().describe()

In [ ]:
# Display the full cleaned dataset
df

### 4.4 Verify Normalization

In [ ]:
# Sanity check: verify normalization worked correctly
# Each experiment should have mean ≈ 0 and std ≈ 1
print("Normalized scores per experiment (first 10):")
normalization_check = df.groupby('source_file')['score_norm'].agg(['mean', 'std'])
print(normalization_check.head(10))

print("\n✓ Normalization successful!" if (abs(normalization_check['mean'].mean()) < 0.01) else "⚠ Check normalization")

### 4.5 Save Cleaned Dataset

Let's save our cleaned and normalized data for future use.

In [ ]:
# Save cleaned dataset to CSV
output_dir = BASE_DIR / "ProteinGym_DMS_data"
output_file = output_dir / "cleaned_ProteinGym_DMS_substitutions.csv"

# Ensure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Save to CSV
df.to_csv(output_file, index=False)

print(f"✓ Cleaned dataset saved to: {output_file}")
print(f"  Total rows: {len(df):,}")
print(f"  Total columns: {len(df.columns)}")

## 🎯 Ready for Machine Learning!

Your cleaned dataset is now ready to use with protein language models:

**Options:**
- Feed `mut_seq` (mutated sequences) directly to models like **ESM-2**, **ProteinBERT**, or **ProtT5**
- Use `(wildtype_sequence, mutation)` pairs for models that support variant input
- Use `score_norm` as regression targets
- Use `score_bin` for classification tasks

---

## 📊 Step 5: Data Visualization

Let's explore our cleaned dataset visually to understand its characteristics and identify potential issues.

### 5.1 Setup Visualization Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for prettier plots
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

# Quick sanity check
print("Dataset Summary Statistics:")
print(df.describe())
print("\n" + "="*50)

### 5.2 DMS Score Distributions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw DMS scores
axes[0].hist(df['score'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_title("Distribution of Raw DMS Scores", fontsize=14, fontweight='bold')
axes[0].set_xlabel("DMS Score", fontsize=12)
axes[0].set_ylabel("Frequency", fontsize=12)
axes[0].grid(alpha=0.3)

# Normalized DMS scores
axes[1].hist(df['score_norm'], bins=50, color='seagreen', alpha=0.7, edgecolor='black')
axes[1].set_title("Distribution of Normalized DMS Scores", fontsize=14, fontweight='bold')
axes[1].set_xlabel("Normalized Score (z-score)", fontsize=12)
axes[1].set_ylabel("Frequency", fontsize=12)
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Mean = 0')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Score distributions look good! Normalization has centered the data around 0.")

### 5.3 Mutation Type Analysis

In [ ]:
# Extract mutation type (e.g., 'A123C' → 'AC')
df['mutation_type'] = df['mutation'].str.replace(r'[0-9]', '', regex=True)
top_mutations = df['mutation_type'].value_counts().head(20)

plt.figure(figsize=(12, 6))
top_mutations.plot(kind='bar', color='coral', edgecolor='black')
plt.title('Top 20 Most Common Mutation Types', fontsize=16, fontweight='bold')
plt.xlabel("Mutation Type (WT→Mutant)", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✓ Found {df['mutation_type'].nunique()} unique mutation types")

### 5.4 Sequence Length Distribution

In [ ]:
# Calculate sequence lengths
df['seq_length'] = df['mut_seq'].str.len()

plt.figure(figsize=(12, 6))
plt.hist(df['seq_length'], bins=50, color='mediumpurple', alpha=0.7, edgecolor='black')
plt.title("Distribution of Mutated Sequence Lengths", fontsize=16, fontweight='bold')
plt.xlabel("Sequence Length (amino acids)", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.axvline(df['seq_length'].median(), color='red', linestyle='--', linewidth=2, 
            label=f'Median: {df["seq_length"].median():.0f}')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Sequence length stats:")
print(f"  Min: {df['seq_length'].min()}")
print(f"  Max: {df['seq_length'].max()}")
print(f"  Median: {df['seq_length'].median():.0f}")

### 5.5 Wild-Type Amino Acid Distribution

In [ ]:
# Count wild-type amino acids
wt_counts = df['wt_aa'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
wt_counts.plot(kind='bar', color='teal', edgecolor='black')
plt.title('Wild-Type Amino Acid Distribution', fontsize=16, fontweight='bold')
plt.xlabel("Amino Acid (Single Letter Code)", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("✓ All 20 standard amino acids represented!" if len(wt_counts) == 20 else f"Found {len(wt_counts)} amino acids")

---

## 🎉 Tutorial Complete!

**What we accomplished:**
1. ✅ Downloaded and loaded ProteinGym DMS data
2. ✅ Cleaned and standardized the dataset
3. ✅ Parsed mutation strings into components
4. ✅ Normalized DMS scores for ML readiness
5. ✅ Visualized key dataset characteristics

**Next steps:**
- Use this cleaned data with protein language models (ESM-2, ProteinBERT, ProtT5)
- Build predictive models for mutation effects
- Explore uncertainty quantification techniques

---